In [3]:
import os
from getpass import getpass

# Set up OpenAI API key
groq_api_key = getpass("Enter your Groq key: ")
os.environ["GROQ_API_KEY"] = groq_api_key

Enter your Groq key: ··········


In [4]:
!pip install -q langchain openai chromadb tiktoken pypdf langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 24.5 MB/s eta

In [5]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [8]:
!pip install langchain-groq langchain duckduckgo-search langchain_community ddgs  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.6 MB/s eta 0:00:00


In [9]:
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferWindowMemory

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0, max_tokens=1024)
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=5)

/tmp/ipython-input-2719473669.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=5)


In [11]:
loader = PyPDFLoader("/content/Guidance-Document-on-TB-Mukt-Bharat-Abhiyan_0.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [12]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # lightweight and fast
)

/tmp/ipython-input-2627326643.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
pip install langchain faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.6 MB/s eta 0:00:00


In [14]:
vectorstore = FAISS.from_documents(chunks, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [17]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

system_template = """
You are a TB awareness assistant.
Answer ONLY using the information in the provided source document chunks.
- Always cite the source chunk id (metadata 'source').
- If the answer is not in the document, say: "I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666."
- Do not make up or assume anything outside the document.

Examples (do NOT invent answers):
Q: Who is Sachin Tendulkar?
A: I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666.

Q: What are TB symptoms?
A: The main symptoms are: persistent cough for 2 weeks or more, fever, night sweats, weight loss, fatigue. Source: chunk_12
"""

human_template = """
Use the following document snippets to answer the question.
If snippets do not contain the answer, reply: "I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666."

--- Retrieved snippets:
{context}

--- Question:
{question}
"""

system_msg = SystemMessagePromptTemplate.from_template(system_template)
human_msg = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

In [18]:
from langchain.chains import RetrievalQA

system_instructions = """
You are TB-AwareGPT, an assistant for NGO volunteers.
Answer using ONLY the provided manual excerpts. Do NOT provide medical advice beyond the manual.
If the question is not related to TB, please say "Question not related to TB, please ask question related to TB"
When volunteers prefer Hindi, produce answers in Hindi. Keep replies short and actionable.
"""

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": chat_prompt},
)

In [19]:
def answer_query(user_name, question, lang="en"):
    # get retrieved docs and answer
    result = qa_chain({"query": question})
    answer = result["result"]  # the LLM-produced answer
    # Localization
    if lang.lower().startswith("hi"):
        # ask LLM to translate/compose in Hindi, but still constrained
        prompt = f"Translate and adapt to conversational Hindi the following factual answer (do not add new info):\n\n{answer}"
        answer_hi = llm(prompt)
        return answer_hi.content
    return answer

In [20]:
def generate_quiz(section_text, n_questions=3, lang="en"):
    prompt = f"""
    Create {n_questions} short quiz items (question, three options, correct option, one-line explanation)
    based ONLY on the following text: {section_text}
    Output as JSON list: [{{"q":"", "opts":["","", ""], "a":0, "ex":"..."}}, ...]
    """
    resp = llm(prompt)
    # parse resp.content into structured quiz (use json.loads after ensuring valid JSON)
    return resp.content

def grade_answer(quiz_item, given_answer_index):
    correct = quiz_item["a"]
    if given_answer_index == correct:
        return True, "Correct: " + quiz_item["ex"]
    return False, f"Incorrect. Correct: option {correct+1}. {quiz_item['ex']}"

In [21]:
query = "What are the common symptoms of TB I should tell people about?"
result = qa_chain.invoke({"query": query})
print("Answer:", result["result"])

Answer: The main symptoms are: 
- Cough for more than 2 weeks 
- Fever 
- Chest pain 
- Blood in sputum 
- Night sweats 
- Loss of appetite 
- Weight loss 
- Fatigue 
- Swelling in the neck 
- Joint pain or backache 
- Headache or confusion 
- Blood in urine 
- Abdominal pain or constipation. 
Source: chunk_1, chunk_12, chunk_16


In [22]:
query = "Who is Sachin Tendulkar"
result = qa_chain.invoke({"query": query})
print("Answer:", result["result"])

Answer: I cannot find that in the manual; please refer to the local PHC or call 1800-11-6666.
